In [8]:
import pandas as pd

from unsloth.utils.profiling import get_events_df, TorchOpNode
import json


from functools import partial
import torch
from torch.profiler._memory_profiler import (
    _CATEGORY_TO_INDEX,
    _CATEGORY_TO_COLORS,
    _ACTION_TO_INDEX,
)
from torch.profiler._memory_profiler import Category, Action

INDEX_TO_ACTION = {v: k for k, v in _ACTION_TO_INDEX.items()}
ACTION_TO_NAME = {v: k for k, v in Action._member_map_.items()}
INDEX_TO_CATEGORY = {v: k for k, v in _CATEGORY_TO_INDEX.items()}
CATEGORY_TO_NAME = {v: k for k, v in Category._member_map_.items()}

In [9]:
mem = torch.load("/home/jeromeku/Cpp/CUDA/unsloth-dev/tests/fused_cel/mem_test/out.pt")

In [10]:
mem_timeline = torch.load(
    "/home/jeromeku/Cpp/CUDA/unsloth-dev/tests/fused_cel/mem_test/mem_test-05_15-memory-timeline.pt"
)
# raw_mem_timeline = json.load(
#     open(
#         "/home/jeromeku/Cpp/CUDA/unsloth-dev/tests/fused_cel/flop_prof/shift-05_15-memory-timeline_raw.json"
#     )
# )
# events = torch.load("./flop_prof/shift-05_15-events.pt")
# event_avgs = torch.load("./flop_prof/shift-05_15-key_averages.pt")

In [11]:
def add_category_to_tl(x, categories):
    ts, action, (key, version), num_bytes = x

    id = key.id if hasattr(key, "id") else None
    storage = key.storage if hasattr(key, "storage") else None
    return (
        ts,
        key,
        id,
        storage,
        key.device.type,
        version,
        action,
        categories.get(key, version),
        num_bytes,
    )


header = [
    "ts",
    "key",
    "id",
    "storage",
    "device",
    "version",
    "action_enum",
    "category_enum",
    "num_bytes",
]


full_tl = list(
    map(
        partial(add_category_to_tl, categories=mem_timeline.categories),
        mem_timeline.timeline,
    )
)
df = pd.DataFrame(full_tl, columns=header)
df["category"] = df["category_enum"].map(CATEGORY_TO_NAME).fillna("UNKNOWN")
df["action"] = df["action_enum"].map(ACTION_TO_NAME).fillna("UNKNOWN")

In [13]:
df.head()

,ts,key,id,storage,device,version,action_enum,category_enum,num_bytes,category,action
0,1715808038469504468,id=0: 0x302000000 (1) (cuda:0),0.0,0x302000000 (1),cuda,0,Action.CREATE,Category.INPUT,1024,INPUT,CREATE
1,1715808038471083161,id=1: 0x302000400 (2) (cuda:0),1.0,0x302000400 (2),cuda,0,Action.CREATE,Category.INPUT,8192,INPUT,CREATE
2,1715808038471371912,id=2: 0x302002400 (3) (cuda:0),2.0,0x302002400 (3),cuda,0,Action.CREATE,Category.INPUT,4096,INPUT,CREATE
3,1715808038482300713,"Key(device=device(type='cuda', index=0))",NaN,None,cuda,0,Action.CREATE,None,8519680,UNKNOWN,CREATE
4,1715808038485574764,id=3: 0x302003400 (5) (cuda:0),3.0,0x302003400 (5),cuda,0,Action.CREATE,Category.INPUT,512,INPUT,CREATE


In [15]:
from dataclasses import asdict

mem_dict = [asdict(o) for o in mem]

In [16]:
mem_df = pd.DataFrame.from_dict(mem_dict)

In [22]:
mem_df

,name,inputs,outputs
0,-- Initialization ----------------------------...,(),()
1,aten::randn,[],"[({'device': cuda:0, 'id': 0, 'storage': {'ptr..."
2,aten::randn,[],"[({'device': cuda:0, 'id': 1, 'storage': {'ptr..."
3,-- Matmul ------------------------------------...,(),()
4,aten::mm,"[({'device': cuda:0, 'id': 0, 'storage': {'ptr...","[({'device': cuda:0, 'id': 2, 'storage': {'ptr..."
5,-- Backward ----------------------------------...,(),()
6,aten::sum,"[({'device': cuda:0, 'id': 2, 'storage': {'ptr...","[({'device': cuda:0, 'id': 3, 'storage': {'ptr..."
7,aten::ones_like,"[({'device': cuda:0, 'id': 3, 'storage': {'ptr...","[({'device': cuda:0, 'id': 4, 'storage': {'ptr..."
8,aten::expand,"[({'device': cuda:0, 'id': 4, 'storage': {'ptr...","[({'device': cuda:0, 'id': 4, 'storage': {'ptr..."
9,aten::t,"[({'device': cuda:0, 'id': 0, 'storage': {'ptr...","[({'device': cuda:0, 'id': 0, 'storage': {'ptr..."


In [21]:
df

,ts,key,id,storage,device,version,action_enum,category_enum,num_bytes,category,action
0,1715808038469504468,id=0: 0x302000000 (1) (cuda:0),0.0,0x302000000 (1),cuda,0,Action.CREATE,Category.INPUT,1024,INPUT,CREATE
1,1715808038471083161,id=1: 0x302000400 (2) (cuda:0),1.0,0x302000400 (2),cuda,0,Action.CREATE,Category.INPUT,8192,INPUT,CREATE
2,1715808038471371912,id=2: 0x302002400 (3) (cuda:0),2.0,0x302002400 (3),cuda,0,Action.CREATE,Category.INPUT,4096,INPUT,CREATE
3,1715808038482300713,"Key(device=device(type='cuda', index=0))",NaN,None,cuda,0,Action.CREATE,None,8519680,UNKNOWN,CREATE
4,1715808038485574764,id=3: 0x302003400 (5) (cuda:0),3.0,0x302003400 (5),cuda,0,Action.CREATE,Category.INPUT,512,INPUT,CREATE
5,1715808038492845914,id=4: 0x302003600 (6) (cuda:0),4.0,0x302003600 (6),cuda,0,Action.CREATE,Category.INPUT,512,INPUT,CREATE
6,1715808038496427044,id=5: 0x302003800 (7) (cuda:0),5.0,0x302003800 (7),cuda,0,Action.CREATE,Category.GRADIENT,8192,GRADIENT,CREATE
7,1715808038496456301,id=6: 0x302005800 (8) (cuda:0),6.0,0x302005800 (8),cuda,0,Action.CREATE,Category.TEMPORARY,4096,TEMPORARY,CREATE
8,1715808038505211371,"Key(device=device(type='cuda', index=0))",NaN,None,cuda,0,Action.CREATE,None,8519680,UNKNOWN,CREATE
9,1715808038507498821,id=6: 0x302005800 (8) (cuda:0),6.0,0x302005800 (8),cuda,0,Action.DESTROY,Category.TEMPORARY,4096,TEMPORARY,DESTROY


In [104]:
def mem_to_action_category(x):
    ts, action_idx, num_bytes, category_idx = x
    return ts, INDEX_TO_ACTION[action_idx], num_bytes, INDEX_TO_CATEGORY[category_idx]


tensor_map = list(
    map(lambda x: mem_to_action_category(x), raw_mem_timeline)
)  # raw_mem_timeline

In [107]:
events_df = get_events_df(events)

In [114]:
mm_events = events_df[events_df["key"].str.contains("aten::mm")]

In [124]:
concretes = events_df.iloc[[any(l) for l in events_df.concrete_inputs.to_list()]]

In [126]:
concretes.key

2                  aten::randn
3                  aten::empty
6                aten::normal_
12                 aten::randn
13                 aten::empty
                ...           
992          aten::result_type
994        aten::_foreach_add_
995          aten::result_type
997    aten::_foreach_addcdiv_
998          aten::result_type
Name: key, Length: 391, dtype: object

In [97]:
g = cuda.groupby("category")

In [99]:
g["num_bytes"].sum()

category
ACTIVATION         3927055360
AUTOGRAD_DETAIL    5893474304
GRADIENT            166461440
INPUT                  614400
OPTIMIZER_STATE     278528000
PARAMETER           180224000
UNKNOWN             344719360
Name: num_bytes, dtype: int64

{('cpu', 'OPTIMIZER_STATE'): [29, 30, 35, 76, 117, 158, 199], ('cpu', 'UNKNOWN'): [33, 34, 36, 74, 75, 77, 115, 116, 118, 156, 157, 159, 197, 198, 200], ('cuda', 'ACTIVATION'): [3, 5, 7, 8, 9, 10, 11, 12, 15, 18, 28, 47, 48, 49, 51, 52, 53, 54, 55, 56, 59, 62, 73, 88, 89, 90, 92, 93, 94, 95, 96, 97, 100, 103, 114, 129, 130, 131, 133, 134, 135, 136, 137, 138, 141, 144, 155, 170, 171, 172, 174, 175, 176, 177, 178, 179, 182, 185, 196, 211], ('cuda', 'AUTOGRAD_DETAIL'): [13, 16, 17, 19, 20, 21, 22, 23, 27, 57, 60, 61, 63, 64, 65, 66, 67, 69, 70, 72, 98, 101, 102, 104, 105, 106, 107, 108, 110, 111, 113, 139, 142, 143, 145, 146, 147, 148, 149, 151, 152, 154, 180, 183, 184, 186, 187, 188, 189, 190, 192, 193, 195], ('cuda', 'GRADIENT'): [24, 26, 46, 68, 71, 87, 109, 112, 128, 150, 153, 169, 191, 194, 210], ('cuda', 'INPUT'): [0, 2, 6, 14, 50, 58, 91, 99, 132, 140, 173, 181], ('cuda', 'OPTIMIZER_STATE'): [31, 32, 38, 39, 40, 79, 80, 81, 120, 121, 122, 161, 162, 163, 202, 203, 204], ('cuda', 'PA

In [71]:
k = df.key.iloc[0]

In [79]:
k.id

0

In [66]:
coalesced = mem_timeline._coalesce_timeline("cuda:0")

In [68]:
aggregates = [[INDEX_TO_CATEGORY[c] for c in x] for x in coalesced[1]]

KeyError: 524288

In [12]:
allocs_by_category = mem_timeline._coalesce_timeline("cuda:0")

[0, 16384000, 32768000, 532480, 0, 0, 524288, 0, 17039360]

In [23]:
_CATEGORY_TO_INDEX

{<Category.PARAMETER: 6>: 0,
 <Category.OPTIMIZER_STATE: 7>: 1,
 <Category.INPUT: 1>: 2,
 <Category.TEMPORARY: 2>: 3,
 <Category.ACTIVATION: 3>: 4,
 <Category.GRADIENT: 4>: 5,
 <Category.AUTOGRAD_DETAIL: 5>: 6,
 None: 7}

In [24]:
_ACTION_TO_INDEX

{<Action.PREEXISTING: 1>: 1,
 <Action.CREATE: 2>: 2,
 <Action.INCREMENT_VERSION: 3>: 3,
 <Action.DESTROY: 4>: 4}